# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS-109A Introduction to Data Science 


## Fine Particulate Air Pollution and COVID-19

**Harvard University**<br>
**Spring 2020**<br>
Jack Luby, Hakeem Angulu, and Louie Ayre <br>

---



### The Problem

Fine particulate matter (PM$_{2.5}$) is an air pollutant which has been shown to increase the risk of mortality and hospitalization in exposed populations. 

The fine inhalable particles of PM$_{2.5}$ impact communitites at the local level, incrementally decreasing their life expectancies as ambient concentrations of the pollutant rise. Despite their adverse effects, concentrations of PM$_{2.5}$ are not well monitored throughout much of the United States (especially in regions of low population density). As a result, many communities (and their care providers) are unaware of the life-shortening ambient concentrations of PM$_{2.5}$ they breathe each day. 

These risks have been heightened by the coronavirus pandemic. COVID-19 is primarily a respiratory disease, and PM$_{2.5}$'s adverse effects on respiratory potential have been theorized and shown to increase the likelihood of developing COVID-19. It is apparent that high concentrations of PM$_{2.5}$ are likely to be founded in rural and low-income communities and communities of color. In addition, the US healthcare and social systems have long underserved those communities. The combination of these factors creates an especially bad prognosis for those communities, and necessitates further study and rapid policy and healthcare interventions.

### Our Project

With PM$_{2.5}$ data and COVID-19 data at hand, this project seeks to understand the relationships between PM$_{2.5}$ pollution, demographic and socioeconomic factors, and COVID-19, with a particular focus on rural and low-income communities, and communities of color.

In [2]:
## Set formatting to CS109 standard
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
# The classics
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

## Import and Prepare Data

### PM_25 Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [4]:
# Import sample airpred data
no_loc_airpred = pd.read_csv("data/airpred.csv")
locations = pd.read_csv("data/airpred_monitor_locations.csv")

In [5]:
# Merge location data
airpred = no_loc_airpred.merge(locations, how='left')

In [6]:
# County PM data
init_county_pm = pd.read_csv("data/county_pm25.csv")

In [7]:
init_county_pm.head()

,fips,year,pm25
0,36103.0,2000,13.749745
1,36103.0,2001,13.681471
2,36103.0,2002,12.549986
3,36103.0,2003,12.436192
4,36103.0,2004,11.738100


In [8]:
# Drop the counties with no data and no fips
county_pm = init_county_pm.dropna()

In [9]:
# Convert fips to int
county_pm['fips'] = county_pm['fips'].astype(int)

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
county_pm.head()

,fips,year,pm25
0,36103,2000,13.749745
1,36103,2001,13.681471
2,36103,2002,12.549986
3,36103,2003,12.436192
4,36103,2004,11.738100


### Socioeconomic, Demographic, and Behavioral Risk Factor Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [11]:
# County socioeconomic and demographic data
init_county_demo = pd.read_csv("data/census_county_interpolated.csv")

In [12]:
# Drop unnecessary axis
init_county_demo.drop(['Unnamed: 0'], axis=1, inplace=True)

In [13]:
init_county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103.0,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103.0,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103.0,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103.0,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103.0,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [14]:
# Drop the counties with no fips
county_demo = init_county_demo.dropna()

In [15]:
# Convert fips to int
county_demo['fips'] = county_demo['fips'].astype(int)

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [17]:
# County behavioral risk factor data
county_brf = pd.read_csv("data/brfss_county_interpolated.csv")

### COVID-19 Data

These data were obtained from the __COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University__, found [here](https://github.com/CSSEGISandData/COVID-19).

In [18]:
def covid_data_prep(date):
    """
    Process COVID-19 county-wise data.
    
    date (string): the date of the data
    """
    init_county_covid = pd.read_csv(f"data/county_covid_{date}.csv")
    
    # Select columns of interest
    select_county_covid = init_county_covid[['FIPS', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
    
    # Add a date column
    select_county_covid['date'] = date
    
    # Drop NA
    select_county_covid.dropna(inplace=True)
    
    # Convert fips to int and format the columns
    select_county_covid['FIPS'] = select_county_covid['FIPS'].astype(int)
    return select_county_covid.rename(str.lower, axis=1)

In [19]:
# Available dates
available_dates = ['03_25', '04_05', '04_15', '04_25', '05_05']

In [20]:
all_county_covid_data = [covid_data_prep(date) for date in available_dates]

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [21]:
all_county_covid_data[-1]

,fips,confirmed,deaths,recovered,active,date
0,45001,33,0,0,33,05_05
1,22001,136,10,0,126,05_05
2,51001,429,7,0,422,05_05
3,16001,713,19,0,694,05_05
4,19001,2,0,0,2,05_05
...,...,...,...,...,...,...
2961,99999,103,3,0,100,05_05
2964,66,145,5,0,140,05_05
2991,69,14,2,0,12,05_05
2997,72,1924,99,0,1825,05_05


In [22]:
long_county_covid = pd.concat(all_county_covid_data)

In [23]:
long_county_covid.head()

,fips,confirmed,deaths,recovered,active,date
0,45001,3,0,0,0,03_25
1,22001,2,0,0,0,03_25
2,51001,2,0,0,0,03_25
3,16001,24,0,0,0,03_25
4,19001,1,0,0,0,03_25


### COVID-19 State Policy Data


In [24]:
# Read CSV
init_policy = pd.read_csv("data/state_policy0410.csv")[:-3]
init_policy.head()

,State,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,Closed non-essential businesses,Religious Gatherings Exempt Without Clear Social Distance Mandate*,Alcohol/Liquor Stores Open,Keep Firearms Sellers Open,...,Extend the amount of time an individual can be on unemployment insurance,Paid sick leave,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2016
0,Alabama,3/13/2020,3/18/2020,3/20/2020,3/19/2020,4/4/2020,3/28/2020,0,1.0,1.0,...,0.0,0,93.24,4887871,52420.0,"3,261",5.6,16.8,43.1,"52,466"
1,Alaska,3/11/2020,3/16/2020,0,0,3/28/2020,3/28/2020,0,1.0,1.0,...,0.0,0,1.11,737438,665384.0,"1,907",6.8,10.9,32.8,"4,494"
2,Arizona,3/11/2020,3/16/2020,0,0,3/31/2020,0,1,1.0,1.0,...,0.0,1,62.91,7171646,113990.0,"10,007",5.4,14,39.1,"56,645"
3,Arkansas,3/11/2020,3/17/2020,0,3/13/2020,0,0,0,1.0,1.0,...,0.0,0,56.67,3013825,53179.0,"2,717",4.5,17.2,43.5,"31,756"
4,California,3/4/2020,0,0,0,3/19/2020,0,0,1.0,0.0,...,0.0,1,241.65,39557045,163695.0,"151,278",5.5,12.8,33.3,"262,240"


In [72]:
init_policy.columns

Index(['State', 'State of emergency', 'Date closed K-12 schools',
       'Closed day cares', 'Date banned visitors to nursing homes',
       'Stay at home/ shelter in place', 'Closed non-essential businesses',
       'Religious Gatherings Exempt Without Clear Social Distance Mandate*',
       'Alcohol/Liquor Stores Open', 'Keep Firearms Sellers Open',
       'Closed restaurants except take out', 'Closed gyms',
       'Closed movie theaters', 'Froze evictions',
       'Order freezing utility shut offs', 'Froze mortgage payments',
       'Waived one week waiting period for unemployment insurance',
       'Waive work search requirement for unemployment insurance',
       'Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined',
       'Expand eligibility of unemployment insurance to those who have lost childcare/school closures',
       'Extend the amount of time an individual can be on unemployment insurance',
       'Pai

In [50]:
def days_since(data=init_policy, col='State of emergency'):
    """
    Extract the number of days since policy changes were enacted.
    
    data(DF): the data
    col(str): the column name
    """
    # Initialize days list
    days = []
    
    # Specify date format
    date_format = "%m/%d/%Y"
    
    # Get the current date
    a = dt.today()
    
    # Take the difference
    for date in data[col]:
        if type(date) != str:
            days.append(None)
        elif len(date) < 8:
            days.append(None)
        else:
            b = dt.strptime(date, date_format)
            delta = a - b
            days.append(delta.days)
    return days

In [67]:
# Build State Policy DF
policy = pd.DataFrame(init_policy.copy()['State'])

cols = list(init_policy.columns[1:7]) + list(init_policy.columns[10:17])
for col in cols:
    policy[col] = days_since(col=col)
    
policy.head()

,State,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,Closed non-essential businesses,Closed restaurants except take out,Closed gyms,Closed movie theaters,Froze evictions,Order freezing utility shut offs,Froze mortgage payments,Waived one week waiting period for unemployment insurance
0,Alabama,56,51.0,49.0,50.0,34.0,41.0,50.0,41.0,41.0,NaN,NaN,NaN,53.0
1,Alaska,58,53.0,NaN,NaN,41.0,41.0,51.0,51.0,51.0,NaN,NaN,NaN,43.0
2,Arizona,58,53.0,NaN,NaN,38.0,NaN,49.0,49.0,49.0,45.0,NaN,NaN,51.0
3,Arkansas,58,52.0,NaN,56.0,NaN,NaN,49.0,49.0,NaN,NaN,NaN,NaN,52.0
4,California,65,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,42.0,NaN,NaN,57.0


## Note: 

Some data that are missing may be worth inserting manually, namely CA for K-12

'Waived one week waiting period for unemployment insurance' includes binary and dates, indicating that for some states policy changes are known to have been enacted but the dates are missing.

I'm having trouble converting floats to ints for some reason.

In [71]:
# Use to 'NCHSURCodes2013.csv' and 'statecode' to link other DF to State Policy via fips
pd.read_csv("data/NCHSURCodes2013.csv").columns

Index(['FIPS', 'State Abr.', 'County name', 'CBSA title', 'CBSA 2012 pop',
       'County 2012 pop', '2013 code', '2006 code', '1990-based code',
       'Unnamed: 9'],
      dtype='object')

### 